### 오피스텔 500m 최종 파일 파일

In [5]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

data_zip_path = '/content/drive/MyDrive/오피스텔_500m_최종.csv'
extract_dir = '/오피스텔_500m_최종'

data = pd.read_csv(data_zip_path, encoding='euc-kr')

data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,NO,시군구,단지명,도로명,위도,경도,bus_count_500m,CCTV_count,school_count,streetlight_count
0,1,서울특별시 광진구 화양동,광진청년생활,동일로30길 31,37.547678,127.069085,10,208,1,7
1,5,서울특별시 동대문구 전농동,청량리 창보 리버리치2,왕산로 192,37.579581,127.044612,14,220,6,0
2,9,서울특별시 종로구 숭인동,블루카운티 숭인Ⅰ,종로62길 23,37.572843,127.019649,5,256,107,0
3,10,서울특별시 종로구 효제동,BS타워,종로35길 12,37.571758,127.004086,12,148,1,0
4,11,서울특별시 성동구 성수동2가,성원상떼뷰오피스텔,아차산로15길 10,37.543696,127.062516,6,178,1,0


### 다중회귀분석을 통한 안점도 점수 계산
- 4개 컬럼 가중치 1:1:1:1
- 절편=0
- 최저점을 50, 최고점을 100으로 고정

In [17]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from google.colab import drive

drive.mount('/content/drive')

data_file = "/content/drive/MyDrive/오피스텔_500m_최종.csv"  # Google Drive 파일 경로
df = pd.read_csv(data_file, encoding='euc-kr')

columns = ['bus_count_500m', 'CCTV_count', 'school_count', 'streetlight_count']
X = df[columns]

scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

model = LinearRegression(fit_intercept=False)
model.fit(X_normalized, [1] * len(X_normalized))

df['safety_score_raw'] = X_normalized @ model.coef_

df['safety_score_normalized'] = (df['safety_score_raw'] - df['safety_score_raw'].min()) / \
                                (df['safety_score_raw'].max() - df['safety_score_raw'].min())

df['safety_score'] = df['safety_score_normalized'] * 50 + 50

output_file = "/content/drive/MyDrive/오피스텔_safety_scores_scaled_50_100.csv"  # Google Drive 저장 경로
df.to_csv(output_file, index=False)

print(f"안전도 점수가 계산되었습니다. 파일이 다음 위치에 저장되었습니다: {output_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
안전도 점수가 계산되었습니다. 파일이 다음 위치에 저장되었습니다: /content/drive/MyDrive/오피스텔_safety_scores_scaled_50_100.csv


### 최저점, 최고점 확인

In [18]:
min_safety_score = df['safety_score'].min()
max_safety_score = df['safety_score'].max()

print(f"최저 안전도 점수: {min_safety_score}")
print(f"최고 안전도 점수: {max_safety_score}")


최저 안전도 점수: 50.0
최고 안전도 점수: 100.0


### 최저점, 최고점 매물 개수 확인

In [20]:
min_safety_count = df[df['safety_score'] == df['safety_score'].min()].shape[0]
max_safety_count = df[df['safety_score'] == df['safety_score'].max()].shape[0]

print(f"최저점 매물 개수: {min_safety_count}")
print(f"최고점 매물 개수: {max_safety_count}")


최저점 매물 개수: 5
최고점 매물 개수: 1


### 만들어진 매물별 안전도점수 파일 확인

In [22]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

data_zip_path = '/content/drive/MyDrive/오피스텔_safety_scores_scaled_50_100.csv'
extract_dir = '/오피스텔_safety_scores_scaled_50_100'

data = pd.read_csv(data_zip_path)

data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,NO,시군구,단지명,도로명,위도,경도,bus_count_500m,CCTV_count,school_count,streetlight_count,safety_score_raw,safety_score_normalized,safety_score
0,1,서울특별시 광진구 화양동,광진청년생활,동일로30길 31,37.547678,127.069085,10,208,1,7,0.778165,0.323115,66.155746
1,5,서울특별시 동대문구 전농동,청량리 창보 리버리치2,왕산로 192,37.579581,127.044612,14,220,6,0,0.958029,0.398260,69.913016
2,9,서울특별시 종로구 숭인동,블루카운티 숭인Ⅰ,종로62길 23,37.572843,127.019649,5,256,107,0,1.172307,0.487783,74.389167
3,10,서울특별시 종로구 효제동,BS타워,종로35길 12,37.571758,127.004086,12,148,1,0,0.726021,0.301329,65.066475
4,11,서울특별시 성동구 성수동2가,성원상떼뷰오피스텔,아차산로15길 10,37.543696,127.062516,6,178,1,0,0.558603,0.231384,61.569202


### 동 컬럼을 추가해서 클러스터링 결과(0, 1)컬럼 붙이기

In [29]:
clustering_data.rename(columns={'dong_name': '동'}, inplace=True)

merged_df = pd.merge(df, clustering_data, on='동', how='left')

output_file_with_clustering = "/content/drive/MyDrive/오피스텔_안전도점수_클러스터링_병합.csv"
merged_df.to_csv(output_file_with_clustering, index=False)

print(f"병합된 데이터가 다음 경로에 저장되었습니다: {output_file_with_clustering}")



병합된 데이터가 다음 경로에 저장되었습니다: /content/drive/MyDrive/오피스텔_safety_scores_with_clustering.csv


### 매물별 안전도 점수와 클러스터링 결과 병합 파일 확인

In [31]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

data_zip_path = '/content/drive/MyDrive/오피스텔_안전도점수_클러스터링_병합.csv'
extract_dir = '/오피스텔_안전도점수_클러스터링_병합'

data = pd.read_csv(data_zip_path, encoding='euc-kr')

data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,NO,시군구,단지명,도로명,위도,경도,bus_count_500m,CCTV_count,school_count,streetlight_count,safety_score_raw,safety_score_normalized,safety_score,동,cluster
0,1,서울특별시 광진구 화양동,광진청년생활,동일로30길 31,37.547678,127.069085,10,208,1,7,0.778165,0.323115,66.155745,화양동,0
1,5,서울특별시 동대문구 전농동,청량리 창보 리버리치2,왕산로 192,37.579581,127.044612,14,220,6,0,0.958029,0.398260,69.913016,전농동,0
2,9,서울특별시 종로구 숭인동,블루카운티 숭인Ⅰ,종로62길 23,37.572843,127.019649,5,256,107,0,1.172307,0.487783,74.389167,숭인동,0
3,10,서울특별시 종로구 효제동,BS타워,종로35길 12,37.571758,127.004086,12,148,1,0,0.726021,0.301329,65.066475,효제동,0
4,11,서울특별시 성동구 성수동2가,성원상떼뷰오피스텔,아차산로15길 10,37.543696,127.062516,6,178,1,0,0.558603,0.231384,61.569202,성수동2가,0


### 매물별 안전도 점수 클러스터링 결과에 따라 조정
- cluster:0 -> 그대로
- cluster:1 -> 안전도 점수 * 0.7배로 조정

In [36]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

merged_file = "/content/drive/MyDrive/오피스텔_안전도점수_클러스터링_병합.csv"  # Google Drive 파일 경로
merged_df = pd.read_csv(merged_file, encoding='euc-kr')

merged_df.loc[merged_df['cluster'] == 1, 'safety_score'] *= 0.7

output_file_adjusted = "/content/drive/MyDrive/오피스텔_안전도_클러스터링_조정.csv"
merged_df.to_csv(output_file_adjusted, index=False)

print(f"조정된 데이터가 다음 경로에 저장되었습니다: {output_file_adjusted}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
조정된 데이터가 다음 경로에 저장되었습니다: /content/drive/MyDrive/오피스텔_안전도_클러스터링_조정.csv


### 최저점, 최고점 확인

In [38]:
import pandas as pd

adjusted_file = "/content/drive/MyDrive/오피스텔_안전도_클러스터링_조정.csv"
adjusted_df = pd.read_csv(adjusted_file)

min_safety_score = adjusted_df['safety_score'].min()
max_safety_score = adjusted_df['safety_score'].max()

print(f"최저 안전도 점수: {min_safety_score}")
print(f"최고 안전도 점수: {max_safety_score}")


최저 안전도 점수: 37.728208574
최고 안전도 점수: 100.0


In [39]:
min_safety_count = adjusted_df[adjusted_df['safety_score'] == adjusted_df['safety_score'].min()].shape[0]
max_safety_count = adjusted_df[adjusted_df['safety_score'] == adjusted_df['safety_score'].max()].shape[0]

print(f"최저 안전도 점수 개수: {min_safety_count}")
print(f"최고 안전도 점수 개수: {max_safety_count}")


최저 안전도 점수 개수: 1
최고 안전도 점수 개수: 1
